# Case Study - Recommender EDA

In [14]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import numpy as np
import pandas as pd
import scipy.stats as scs
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
from bs4 import BeautifulSoup as bs

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import pos_tag
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [15]:
# Target output
target = pd.read_csv('data/sample_submission.csv')
target.head()

,user_id,joke_id,rating
0,49541,113,5
1,39499,37,5
2,36804,73,5
3,16470,33,5
4,22614,128,5


In [16]:
ratings_df = pd.read_table('data/ratings.dat')
ratings_df.head()

,user_id,joke_id,rating
0,34888,111,-9.71875
1,49651,119,4.75000
2,29027,137,2.62500
3,25235,30,-2.43750
4,50499,19,2.18750


In [5]:
jokes = []
joke = ''
with open('data/jokes.txt') as f:

    for line in f:
        line_list = line.split(' ')
        if line_list[0][-1:] == ':':
            jokes.append(joke)    # add the last joke into the jokes list as a new joke has begun 
            joke = ' '.join(line_list[1:])
        else:
            joke += line
jokes.append(joke)
jokes = jokes[1:]
print(jokes[5])    

Bill and Hillary Clinton are on a trip back to Arkansas. They're almost out of gas, so Bill pulls into a service station on the outskirts of town. The attendant runs out of the station to serve them when Hillary realizes it's an old boyfriend from high school. She and the attendant chat as he gases up their car and cleans the windows. Then they all say goodbye.

As Bill pulls the car onto the road, he turns to Hillary and says, "Now aren't you glad you married me and not him? You could've been the wife of a grease monkey!"

To which Hillary replied, "No, Bill. If I would have married him, you'd be pumping gas and he would be the President!" 



In [ ]:
# filename = 'data/jokes.dat'
# html = open(filename,'r').read()
# data = bs(html, 'html.parser')

# jokes = [joke.text.replace('\n', '').replace('\r', '') for joke in data.select('p')]

In [ ]:
jokes[0]

In [17]:
import pandas as pd
import graphlab as gl
from bs4 import BeautifulSoup


def add_joke_len(filename):
    r = open(filename)
    soup = BeautifulSoup(r, 'html.parser')

    joke_list = soup.find_all('p')

    joke_len_list = []
    breaks_list = []
    q_list = []

    for x in joke_list:
        q = 0
        breaks = len(x.find_all('br'))
        breaks_list.append(breaks)
        joke_len = 0
        for y in x:
            joke_len += len(y)
            if y.find('?') > 0:
                q = 1
        joke_len_list.append(joke_len)
        q_list.append(q)

    return q_list, breaks_list, joke_len_list



sample_sub_fname = "data/sample_submission.csv"
ratings_data_fname = "data/ratings.dat"
output_fname = "data/test_ratings.csv"

filename = 'data/jokes.dat'
q_list, breaks_list, joke_len = add_joke_len(filename)

joke_info = gl.SFrame({'joke_id': range(1,151), 'joke_len': joke_len, 'joke_break': breaks_list, 'question': q_list})

ratings = gl.SFrame(ratings_data_fname, format='tsv')
sample_sub = pd.read_csv(sample_sub_fname)
for_prediction = gl.SFrame(sample_sub)
rec_engine = gl.factorization_recommender.create(observation_data=ratings,
                                                 user_id="user_id",
                                                 item_id="joke_id",
                                                 target='rating',
                                                 item_data = joke_info,
                                                 solver='auto')

sample_sub.rating = rec_engine.predict(for_prediction)
sample_sub.to_csv(output_fname, index=False)


Finished parsing file /Users/souljourner/Library/Mobile Documents/com~apple~CloudDocs/Development/Data Science/zipfian/case-study-recommender/data/ratings.dat

Parsing completed. Parsed 100 lines in 0.364178 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,int,float]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /Users/souljourner/Library/Mobile Documents/com~apple~CloudDocs/Development/Data Science/zipfian/case-study-recommender/data/ratings.dat

Parsing completed. Parsed 1218325 lines in 0.577221 secs.

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 10.6

Final training RMSE: 3.25568

In [15]:
gl.factorization_recommender.create?

In [ ]:
import pandas as pd
import numpy as np
import graphlab as gl
from bs4 import BeautifulSoup
from pprint import pprint


def add_joke_len(filename):
    r = open(filename)
    soup = BeautifulSoup(r, 'html.parser')

    joke_list = soup.find_all('p')

    joke_len_list = []
    q_list = []
    clinton_list = []

    for x in joke_list:
        if x.find('?') > 0:
            q = 1
        else:
            q = 0
        if x.find('Clinton') > 0:
            clinton = 1
        else:
            clinton = 0
        joke_len_list.append(np.log(len(x)))
        q_list.append(q)
        clinton_list.append(clinton)

    return q_list, joke_len_list, clinton_list


sample_sub_fname = "data/sample_submission.csv"
ratings_data_fname = "data/ratings.dat"
output_fname = "data/test_ratings.csv"
test_set = "data/validation_data.csv"

filename = 'data/jokes.dat'
q_list, joke_len, clinton_list = add_joke_len(filename)

#joke_info = gl.SFrame({'joke_id': range(1,151), 'joke_len': joke_len, 'question': q_list})
joke_info = gl.SFrame({'joke_id': range(1,151),  'question': q_list, 'clinton': clinton_list})

ratings = gl.SFrame(ratings_data_fname, format='tsv')
sample_sub = pd.read_csv(sample_sub_fname)
for_prediction = gl.SFrame(sample_sub)
rec_engine = gl.factorization_recommender.create(observation_data=ratings,
                                                 user_id="user_id",
                                                 item_id="joke_id",
                                                 target='rating',
                                                 item_data = joke_info,
                                                 solver='auto',
                                                 regularization=0)

sample_sub.rating = rec_engine.predict(for_prediction)
sample_sub.rating = [min(max(x, -10),10) for x in sample_sub.rating]
sample_sub.to_csv(output_fname, index=False)

print("cross validation")
kfolds = gl.cross_validation.KFold(ratings, 5)
params = dict(user_id='user_id', item_id='joke_id', target='rating',
                  solver='auto', side_data_factorization=False)

print("parameter search")
paramsearch = gl.model_parameter_search.create(
                    kfolds,
                    gl.recommender.factorization_recommender.create,
                    params)

print "best params by rmse:"
pprint(paramsearch.get_best_params('mean_validation_rmse'))


Finished parsing file /Users/souljourner/Library/Mobile Documents/com~apple~CloudDocs/Development/Data Science/zipfian/case-study-recommender/data/ratings.dat

Parsing completed. Parsed 100 lines in 0.370777 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,int,float]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /Users/souljourner/Library/Mobile Documents/com~apple~CloudDocs/Development/Data Science/zipfian/case-study-recommender/data/ratings.dat

Parsing completed. Parsed 1218325 lines in 0.571563 secs.

...

Final training RMSE: 3.00674

[INFO] graphlab.deploy.job: Validating job.
[INFO] graphlab.deploy.job: Creating a LocalAsync environment called 'async'.
[INFO] graphlab.deploy.map_job: Validation complete. Job: 'Model-Parameter-Search-Mar-07-2017-14-34-4600000' ready for execution


cross validation
parameter search


[INFO] graphlab.deploy.map_job: Job: 'Model-Parameter-Search-Mar-07-2017-14-34-4600000' scheduled.
[INFO] graphlab.deploy.job: Validating job.
[INFO] graphlab.deploy.map_job: A job with name 'Model-Parameter-Search-Mar-07-2017-14-34-4600000' already exists. Renaming the job to 'Model-Parameter-Search-Mar-07-2017-14-34-4600000-92fac'.
[INFO] graphlab.deploy.map_job: Validation complete. Job: 'Model-Parameter-Search-Mar-07-2017-14-34-4600000-92fac' ready for execution
[INFO] graphlab.deploy.map_job: Job: 'Model-Parameter-Search-Mar-07-2017-14-34-4600000-92fac' scheduled.
[INFO] graphlab.deploy.job: Validating job.
[INFO] graphlab.deploy.map_job: Validation complete. Job: 'Model-Parameter-Search-Mar-07-2017-14-34-4600001' ready for execution
[INFO] graphlab.deploy.map_job: Job: 'Model-Parameter-Search-Mar-07-2017-14-34-4600001' scheduled.
[INFO] graphlab.deploy.job: Validating job.
[INFO] graphlab.deploy.map_job: Validation complete. Job: 'Model-Parameter-Search-Mar-07-2017-14-34-4600002'

best params by rmse:


In [ ]:
print('hi')

## TF-IDF

In [19]:
# Tokenize and remove stop words

# 1. Create a set of documents.
documents = [joke.lower() for joke in jokes]

# 2. Create a set of tokenized documents.
docs = [word_tokenize(content) for content in documents]

# 3. Strip out stop words from each tokenized document.
stop = set(stopwords.words('english'))
docs = [[word for word in words if word not in stop] for words in docs]


# Stemming / Lemmatization

# 1. Stem using both stemmers and the lemmatizer
porter = PorterStemmer()
snowball = SnowballStemmer('english')
wordnet = WordNetLemmatizer()
docs_porter = [[porter.stem(word) for word in words] for words in docs]
docs_snowball = [[snowball.stem(word) for word in words] for words in docs]
docs_wordnet = [[wordnet.lemmatize(word) for word in words] for words in docs]

# Compare
# for i in xrange(min(len(docs_porter[0]), len(docs_snowball[0]), \
#                                                         len(docs_wordnet[0]))):
#     p, s, w = docs_porter[0][i], docs_snowball[0][i], docs_wordnet[0][i]
#     if len(set((p, s, w))) != 1:
#         print "%s\t%s\t%s\t%s" % (docs[0][i], p, s, w)


# Part of speech tagging

# 1. Create a part of speech tagged version of your already tokenized dataset.

# commented out because it is slow...
#pos_tagged = [pos_tag(tokens) for tokens in docs]

# Bag of words and TF-IDF

# 1. Create vocab, set of unique words
docs = docs_snowball # choose which stemmer/lemmatizer to use
vocab_set = set()
[[vocab_set.add(token) for token in tokens] for tokens in docs]
vocab = list(vocab_set)

# 2. Create word count vectors manually.
matrix = [[0] * len(vocab) for doc in docs]
vocab_dict = dict((word, i) for i, word in enumerate(vocab))
for i, words in enumerate(docs):
    for word in words:
        matrix[i][vocab_dict[word]] += 1

# 3. Create word count vector over the whole corpus.
cv = CountVectorizer(stop_words='english')
vectorized = cv.fit_transform(documents)

tfidf = TfidfVectorizer(stop_words='english')
tfidfed = tfidf.fit_transform(documents)


# Cosine Similarity using TF-IDF

# 1. Compute cosine similarity
cosine_similarities = linear_kernel(tfidfed, tfidfed)

# 2. Print out similarities
# for i, doc1 in enumerate(docs):
#     for j, doc2 in enumerate(docs):
#         print i, j, cosine_similarities[i, j]

In [23]:
np.mean(cosine_similarities, axis=0)

array([ 0.02919092,  0.02759409,  0.00968532,  0.01788923,  0.00682931,
        0.02301323,  0.02429581,  0.01188107,  0.0218484 ,  0.01943399,
        0.01123828,  0.02359791,  0.01254333,  0.02569454,  0.01773403,
        0.00954287,  0.02407927,  0.01385469,  0.01505321,  0.01212943,
        0.01357792,  0.01969422,  0.01108395,  0.00712827,  0.01787093,
        0.01934299,  0.01720096,  0.02890396,  0.01769203,  0.01185635,
        0.01800185,  0.02769295,  0.01833857,  0.01579566,  0.01427675,
        0.02783549,  0.02195491,  0.02377229,  0.02587114,  0.01923327,
        0.01495474,  0.02095016,  0.01703944,  0.02078577,  0.02436381,
        0.02473399,  0.02067519,  0.01776745,  0.02422141,  0.02609303,
        0.01507603,  0.0093577 ,  0.02898229,  0.01656648,  0.01464982,
        0.02632864,  0.00855076,  0.0148265 ,  0.00938203,  0.0181303 ,
        0.02213521,  0.02004861,  0.0212308 ,  0.00749268,  0.02296577,
        0.01610889,  0.01393321,  0.04260604,  0.02617758,  0.01

In [21]:
cosine_similarities.shape

(150, 150)

In [17]:
filename = 'data/jokes.dat'
html = open(filename,'r').read()
data = bs(html, 'html.parser')

jokes = [joke.text.replace('\n', ' ').replace('\r', '').replace('"', '').replace("\\", "").strip() for joke in data.select('p')]

In [18]:
jokes

[u"A man visits the doctor. The doctor says, I have bad news for you. You have cancer and Alzheimer's disease.  The man replies, Well, thank God I don't have cancer!",
 u"This couple had an excellent relationship going until one day he came home from work to find his girlfriend packing. He asked her why she was leaving him and she told him that she had heard awful things about him.   What could they possibly have said to make you move out?  They told me that you were a pedophile.  He replied, That's an awfully big word for a ten year old.",
 u"Q. What's 200 feet long and has 4 teeth?  A. The front row at a Willie Nelson concert.",
 u"Q. What's the difference between a man and a toilet?  A. A toilet doesn't follow you around after you use it.",
 u"Q. What's O. J. Simpson's web address?  A. Slash, slash, backslash, slash, slash, escape.",
 u"Bill and Hillary Clinton are on a trip back to Arkansas. They're almost out of gas, so Bill pulls into a service station on the outskirts of town. T

In [ ]:


import pandas as pd
import numpy as np
import graphlab as gl
from bs4 import BeautifulSoup
from pprint import pprint
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt


def add_joke_len(filename):
    r = open(filename)
    soup = BeautifulSoup(r, 'html.parser')

    joke_list = soup.find_all('p')

    joke_len_list = []
    q_list = []
    clinton_list = []

    for x in joke_list:
        if x.find('?') > 0:
            q = 1
        else:
            q = 0
        if x.find('Clinton') > 0:
            clinton = 1
        else:
            clinton = 0
        joke_len_list.append(np.log(len(x)))
        q_list.append(q)
        clinton_list.append(clinton)

    avg_similarity = np.array([ 0.02919092,  0.02759409,  0.00968532,  0.01788923,  0.00682931,
       0.02301323,  0.02429581,  0.01188107,  0.0218484 ,  0.01943399,
       0.01123828,  0.02359791,  0.01254333,  0.02569454,  0.01773403,
       0.00954287,  0.02407927,  0.01385469,  0.01505321,  0.01212943,
       0.01357792,  0.01969422,  0.01108395,  0.00712827,  0.01787093,
       0.01934299,  0.01720096,  0.02890396,  0.01769203,  0.01185635,
       0.01800185,  0.02769295,  0.01833857,  0.01579566,  0.01427675,
       0.02783549,  0.02195491,  0.02377229,  0.02587114,  0.01923327,
       0.01495474,  0.02095016,  0.01703944,  0.02078577,  0.02436381,
       0.02473399,  0.02067519,  0.01776745,  0.02422141,  0.02609303,
       0.01507603,  0.0093577 ,  0.02898229,  0.01656648,  0.01464982,
       0.02632864,  0.00855076,  0.0148265 ,  0.00938203,  0.0181303 ,
       0.02213521,  0.02004861,  0.0212308 ,  0.00749268,  0.02296577,
       0.01610889,  0.01393321,  0.04260604,  0.02617758,  0.01746225,
       0.01829584,  0.01529294,  0.01799566,  0.02242413,  0.01262908,
       0.03388975,  0.00666667,  0.01243958,  0.01025946,  0.02188953,
       0.02570635,  0.01515648,  0.01025857,  0.01040254,  0.02141484,
       0.01867199,  0.03175679,  0.01994185,  0.0118524 ,  0.0178506 ,
       0.01711281,  0.02184123,  0.0287066 ,  0.022685  ,  0.0094119 ,
       0.01146307,  0.01708327,  0.01328454,  0.01012345,  0.01179002,
       0.01242792,  0.01967614,  0.02329067,  0.01747185,  0.02044013,
       0.0336918 ,  0.0139204 ,  0.02706954,  0.02080146,  0.01493801,
       0.0165347 ,  0.01866338,  0.03047691,  0.01634416,  0.02406709,
       0.0339638 ,  0.02122898,  0.03436507,  0.03301692,  0.01861853,
       0.01405534,  0.01023803,  0.01629764,  0.01589835,  0.02001565,
       0.01207473,  0.03081609,  0.02408324,  0.01490476,  0.0297394 ,
       0.02216062,  0.01174015,  0.03073034,  0.02341649,  0.01462874,
       0.02623898,  0.0155256 ,  0.01611861,  0.01407783,  0.01112362,
       0.00934034,  0.02792675,  0.02102156,  0.02378993,  0.01199531,
       0.01604801,  0.02562478,  0.02435416,  0.01676706,  0.01666902])
    return q_list, joke_len_list, clinton_list, avg_similarity



sample_sub_fname = "data/sample_submission.csv"
ratings_data_fname = "data/ratings.dat"
output_fname = "data/test_ratings.csv"
test_set = "data/validation_data.csv"

filename = 'data/jokes.dat'
q_list, joke_len, clinton_list, avg_similarity = add_joke_len(filename)

#joke_info = gl.SFrame({'joke_id': range(1,151), 'joke_len': joke_len, 'question': q_list})
joke_info = gl.SFrame({'joke_id': range(1,151),  'question': q_list, 'sim': avg_similarity})

ratings = gl.SFrame(ratings_data_fname, format='tsv')
sample_sub = pd.read_csv(sample_sub_fname)
for_prediction = gl.SFrame(sample_sub)
#rec_engine = gl.factorization_recommender.create(observation_data=ratings, user_id="user_id", item_id="joke_id", target='rating', item_data = joke_info, solver='auto', linear_regularization= 1e-09, max_iterations= 50, num_factors= 64, regularization= 0)
rec_engine = gl.factorization_recommender.create(observation_data=ratings, user_id="user_id", item_id="joke_id", target='rating', item_data = joke_info, solver='auto', linear_regularization= 1e-09, max_iterations= 50, num_factors= 10, regularization= 1e-06)


predictions = [min(max(x, -10),10) for x in rec_engine.predict(ratings)]
rmse = np.sqrt(mean_squared_error(ratings['rating'], predictions))
print 'train rmse: {}'.format(rmse)

ratings_test = pd.read_csv(test_set)
for_prediction_test = gl.SFrame(ratings_test)
predictions_test = [min(max(x, -10),10) for x in rec_engine.predict(for_prediction_test)]
test_rmse = np.sqrt(mean_squared_error(ratings_test['rating'], predictions_test))
print 'test rmse: {}'.format(test_rmse)

sample_sub.rating = rec_engine.predict(for_prediction)
sample_sub.rating = [min(max(x, -10),10) for x in sample_sub.rating]
sample_sub.to_csv(output_fname, index=False)


kfolds = gl.cross_validation.KFold(ratings, 5)
params = dict(user_id='user_id', item_id='joke_id', target='rating',
              solver='auto', side_data_factorization=False)

paramsearch = gl.model_parameter_search.create(
                    kfolds,
                    gl.recommender.factorization_recommender.create,
                    params)

print "best params by rmse:"
pprint(paramsearch.get_best_params('mean_validation_rmse'))

Finished parsing file /Users/souljourner/Library/Mobile Documents/com~apple~CloudDocs/Development/Data Science/zipfian/case-study-recommender/data/ratings.dat

Parsing completed. Parsed 100 lines in 0.332726 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,int,float]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /Users/souljourner/Library/Mobile Documents/com~apple~CloudDocs/Development/Data Science/zipfian/case-study-recommender/data/ratings.dat

Parsing completed. Parsed 1218325 lines in 0.579835 secs.

...

Final objective value: 8.38087

Final training RMSE: 2.84259

train rmse: 2.83464168549
test rmse: 4.57643466268


[INFO] graphlab.deploy.job: Validating job.
[INFO] graphlab.deploy.job: Creating a LocalAsync environment called 'async'.
[INFO] graphlab.deploy.map_job: Validation complete. Job: 'Model-Parameter-Search-Mar-07-2017-15-46-5800000' ready for execution
[INFO] graphlab.deploy.map_job: Job: 'Model-Parameter-Search-Mar-07-2017-15-46-5800000' scheduled.
[INFO] graphlab.deploy.job: Validating job.
[INFO] graphlab.deploy.map_job: A job with name 'Model-Parameter-Search-Mar-07-2017-15-46-5800000' already exists. Renaming the job to 'Model-Parameter-Search-Mar-07-2017-15-46-5800000-12d6f'.
[INFO] graphlab.deploy.map_job: Validation complete. Job: 'Model-Parameter-Search-Mar-07-2017-15-46-5800000-12d6f' ready for execution
[INFO] graphlab.deploy.map_job: Job: 'Model-Parameter-Search-Mar-07-2017-15-46-5800000-12d6f' scheduled.
[INFO] graphlab.deploy.job: Validating job.
[INFO] graphlab.deploy.map_job: Validation complete. Job: 'Model-Parameter-Search-Mar-07-2017-15-46-5800001' ready for execution


best params by rmse:
